In [1]:
#导入相关库，并对一些赋予简化命名
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import datasets
from io import StringIO 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
#导入数据
bank = pd.read_csv('./bank.csv', sep=';')
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [3]:
#数据清洗
bank_data = bank.copy()

#对default变量进行处理
bank_data['default_cat'] = bank_data['default'].map({'yes':1,'no':0})
bank_data.drop('default',axis=1,inplace=True)

#bank_data.head() #这里主要测试删除和数据转换操作是否有问题

In [4]:
#处理housing变量
bank_data['housing_cat'] = bank_data['housing'].map({'yes':1,'no':0})
bank_data.drop('housing',axis=1,inplace=True)

#处理loan变量
bank_data['load_cat'] = bank_data['loan'].map({'yes':1,'no':0})
bank_data.drop('loan',axis=1,inplace=True)

#处理y变量
bank_data['deposit_cat'] = bank_data['y'].map({'yes':1,'no':0})
bank_data.drop('y',axis=1,inplace=True)

#处理job变量，合并类别
bank['job'] = bank['job'].replace(['management','admin.'],'white-collar')
bank['job'] = bank['job'].replace(['housemaid','services'],'pink-collar')
bank['job'] = bank['job'].replace(['retired','student','unemployed','unknown'],'other')

#处理poutcome变量中
bank_data['poutcome'] = bank_data['poutcome'].replace(['other'],'unknown')

#删除时间变量
bank_data.drop('month', axis=1, inplace=True)
bank_data.drop('day', axis=1, inplace=True)

#处理pdays变量
bank_data.loc[bank_data['pdays']==-1, 'pdays'] = 10000

#哑变量处理
bank_with_dummies = pd.get_dummies(data = bank_data,
                                    columns = ['job', 'marital', 'education', 'contact', 'poutcome'],
                                    prefix = ['job', 'marital', 'education', 'contact', 'poutcome'])

#bank_with_dummies.head() #测试


,age,balance,duration,campaign,pdays,previous,default_cat,housing_cat,load_cat,deposit_cat,...,education_primary,education_secondary,education_tertiary,education_unknown,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_success,poutcome_unknown
0,30,1787,79,1,10000,0,0,0,0,0,...,True,False,False,False,True,False,False,False,False,True
1,33,4789,220,1,339,4,0,1,1,0,...,False,True,False,False,True,False,False,True,False,False
2,35,1350,185,1,330,1,0,1,0,0,...,False,False,True,False,True,False,False,True,False,False
3,30,1476,199,4,10000,0,0,1,1,0,...,False,False,True,False,False,False,True,False,False,True
4,59,0,226,1,10000,0,0,1,0,0,...,False,True,False,False,False,False,True,False,False,True


In [7]:
#自变量、因变量划分
X = bank_with_dummies.drop('deposit_cat',axis=1)
y = bank_data.deposit_cat

In [9]:
#数据集分割与标准化处理
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.30)

#scaler = StandardScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

In [10]:
#可以简单地调用mlp接口，来测试实战可行性和预测效果情况

#from sklearn.neural_network import MLPClassifier
#from sklearn.metrics import classification_report, confusion_matrix
#mlp = MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=1000)
#mlp.fit(X_train, y_train.values.ravel())

#predictions = mlp.predict(X_test)

#print(confusion_matrix(y_test,predictions))
#print(classification_report(y_test,predictions))



[[1196   28]
 [ 102   31]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      1224
           1       0.53      0.23      0.32       133

    accuracy                           0.90      1357
   macro avg       0.72      0.61      0.64      1357
weighted avg       0.88      0.90      0.89      1357

